In [84]:
import pandas as pd
from pandasql import sqldf

In [2]:
pysqldf = lambda q: sqldf(q, globals())

In [3]:
commutes = pd.read_csv('commutes.csv')

In [4]:
commutes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 8 columns):
id           196 non-null float64
date         196 non-null object
direction    196 non-null object
departure    196 non-null object
arrivale     196 non-null object
cost         196 non-null float64
comfort      196 non-null float64
mode_id      196 non-null float64
dtypes: float64(4), object(4)
memory usage: 12.3+ KB


In [338]:
commutes.head(3)

,id,date,direction,departure,arrivale,cost,comfort,mode_id
0,0.0,2015-01-30 00:00:00,Work-bound,8:25,8:45,5.00,4.0,10.0
1,1.0,2015-01-30 00:00:00,Home-bound,7:05,7:25,5.00,5.0,10.0
2,2.0,2015-02-02 00:00:00,Work-bound,8:26,8:46,10.91,5.0,11.0


This dataset is comprised of two tables, commutes and commute_modes. The `commutes` table has 196 commuter trip records ranging from 1/30/2015 to 2/19/2016. The `commute_modes` table lists 13 commute modes.

In [6]:
basic_commute_stats = commutes.describe()[['cost', 'comfort']]
basic_commute_stats

,cost,comfort
count,196.000000,196.000000
mean,9.031327,3.688776
std,7.697175,1.047623
min,0.000000,1.000000
25%,4.000000,3.000000
50%,7.000000,4.000000
75%,11.650000,4.250000
max,39.760000,5.000000


In [446]:
modes = pd.read_csv('commute_modes.csv')
modes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
id      13 non-null float64
mode    13 non-null object
dtypes: float64(1), object(1)
memory usage: 280.0+ bytes


In [447]:
modes

,id,mode
0,1.0,Curb
1,2.0,Flywheel
2,3.0,Luxor Cab
3,4.0,Lyft
4,5.0,Lyft Line
5,6.0,Muni
6,7.0,Scoot
7,8.0,Sidecar
8,9.0,Uber Pool
9,10.0,UberPool


There are separate records for ‘Uber Pool’ and ‘UberPool’, but a quick internet search shows they are the same thing, and the correct name is ‘UberPool’. The records with mode_id 9 were changed to mode_id 10 and the incorrectly spelled version was dropped.

In [448]:
# Changing mode_id of Uber Pool from 9 to 10 to combine with UberPool, which is the correct name 
commutes['mode_id'] = commutes['mode_id'].apply(lambda x: 10 if x == 9 else x)

In [371]:
# Uber Pool and UberPool are the same thing, so dropping incorrect entry after changing corresponding records in commutes table.
modes = modes[modes['id'] != 9]

In [21]:
# earliest and latest record dates
q1 = """
    SELECT MIN(date) as earliest_record
         , MAX(date) as latest_record
    FROM commutes 
    ;
"""
rec_dates = pysqldf(q1)
rec_dates

,earliest_record,latest_record
0,2015-01-30 00:00:00,2016-02-19 00:00:00


In [374]:
# Joining 2 tables together and adding mode type to group by later
q2 = """
     SELECT c.id AS rec_id
          , c.mode_id
          , m.mode
          , m.mode_type
          , c.date
          , c.direction
          , c.departure
          , c.arrivale
          , c.cost
          , c.comfort
     FROM
       (SELECT * 
           , (CASE 
                 WHEN mode = 'Curb' OR mode = 'Flywheel' OR mode = 'Yellowcab' OR mode = 'Luxor Cab' THEN 'Taxi'
                 WHEN mode = 'Lyft' OR mode = 'Lyft Line' OR mode = 'UberPool' OR mode = 'UberX' OR mode = 'Sidecar' 
                   OR mode = 'Scoot' THEN 'Rideshare'
                 WHEN mode = 'Muni' THEN 'Public transport'
                 ELSE mode
                 END
       ) AS mode_type
       FROM modes) m
     INNER JOIN commutes c
       ON c.mode_id = m.id
     ;
    """
joined = pysqldf(q2)
joined.head()

,rec_id,mode_id,mode,mode_type,date,direction,departure,arrivale,cost,comfort
0,88.0,1.0,Curb,Taxi,2015-07-29 00:00:00,Work-bound,8:36,9:15,33.33,1.0
1,100.0,2.0,Flywheel,Taxi,2015-08-24 00:00:00,Work-bound,8:26,8:43,17.20,5.0
2,103.0,2.0,Flywheel,Taxi,2015-08-27 00:00:00,Home-bound,4:50,5:10,17.85,5.0
3,4.0,4.0,Lyft,Rideshare,2015-02-05 00:00:00,Work-bound,8:44,9:20,21.56,4.0
4,5.0,4.0,Lyft,Rideshare,2015-02-09 00:00:00,Work-bound,8:45,9:09,24.45,4.0


Quick interned seach revealed that commute modes in this dataset fall into 4 different categories: Walk, Public Transport, Rideshare, and Taxi, with vast majority of records being for Rideshare trips.

Basic aggregate statistics show that commute costs in the dataset average at `$9.03` with standard deviation of `$7.7`, so there is pretty wide variation in costs. It becomes more evident when we look closer at average cost for each mode.

In [178]:
# Basic aggregate stats for whole dataset
joined.describe()

,rec_id,mode_id,cost,comfort
count,196.00000,196.000000,196.000000,196.000000
mean,97.50000,7.336735,9.031327,3.688776
std,56.72448,2.648440,7.697175,1.047623
min,0.00000,1.000000,0.000000,1.000000
25%,48.75000,5.000000,4.000000,3.000000
50%,97.50000,7.000000,7.000000,4.000000
75%,146.25000,10.000000,11.650000,4.250000
max,195.00000,13.000000,39.760000,5.000000


In [378]:
# Averages by mode
q6 = """
    SELECT mode
         , AVG(cost) as avg_cost
         , AVG(comfort) as avg_comfort
    FROM joined
    GROUP BY mode
    ORDER BY avg_cost
    ;
     """
avg_by_mode = pysqldf(q6)
avg_by_mode

,mode,avg_cost,avg_comfort
0,Walk,0.000000,3.166667
1,Muni,3.210811,3.027027
2,Scoot,4.181818,2.772727
3,UberPool,6.683333,4.194444
4,Lyft Line,6.787368,3.710526
5,Sidecar,12.762500,3.250000
6,Flywheel,17.525000,5.000000
7,UberX,17.704444,4.555556
8,Yellowcab,18.185000,4.500000
9,Lyft,20.559474,3.894737


In [381]:
# Medians by mode
medians = joined.groupby('mode')[['cost', 'comfort']].median()
medians.sort_values('cost')

,cost,comfort
mode,,
Walk,0.000,3.0
Muni,2.250,3.0
Scoot,4.000,3.0
Lyft Line,7.000,4.0
UberPool,7.000,4.0
Sidecar,13.025,3.0
UberX,16.040,5.0
Flywheel,17.525,5.0
Yellowcab,17.735,4.5


It makes sense that walking would be least expensive, and Taxi services would be most expensive. It was a little surprising, though, that a rideshare service like Lyft was among the most expensive. That could be explained by the majority of the Lyft rides, 16 out of 19, not including Lyft Line,  being work-bound and taken during the rush hour at higher prices.

In [386]:
# Lyft rides distribution by direction
q7 = """
     SELECT direction
          , COUNT(rec_id)
     FROM joined
     WHERE mode = 'Lyft'
     GROUP BY direction
     ;
     """
lyft = pysqldf(q7)
lyft

,direction,COUNT(rec_id)
0,Home-bound,3
1,Work-bound,16


In [375]:
# stats by mode type
q3 = """
    SELECT mode_type
         , COUNT(rec_id) as cnt
         , AVG(cost) as avg_cost
         , MIN(cost) as min_cost
         , MAX(cost) as max_cost
         , AVG(comfort) as avg_comfort
         , MIN(comfort) as min_comfort
         , MAX(comfort) as max_comfort
    FROM joined
    GROUP BY mode_type
    ORDER BY avg_cost
    ;
     """
stats_by_type = pysqldf(q3)
stats_by_type

,mode_type,cnt,avg_cost,min_cost,max_cost,avg_comfort,min_comfort,max_comfort
0,Walk,6,0.000000,0.0,0.00,3.166667,2.0,4.0
1,Public transport,37,3.210811,2.0,25.09,3.027027,1.0,4.0
2,Rideshare,146,10.343973,0.0,39.76,3.856164,1.0,5.0
3,Taxi,7,20.160000,14.0,33.33,4.142857,1.0,5.0


In [324]:
# additional stats by commute mode type
std_dev = joined.groupby('mode_type')[['cost', 'comfort']].std()
std_dev = std_dev.rename(columns=lambda x: x.replace('c', 'std_c')).reset_index()
quant1_cost = joined.groupby('mode_type')[['cost', 'comfort']].quantile(.25)
quant1_cost = quant1_cost.rename(columns=lambda x: x.replace('c', 'q1_c')).reset_index()
quant2_cost = joined.groupby('mode_type')[['cost', 'comfort']].quantile(.5)
quant2_cost = quant2_cost.rename(columns=lambda x: x.replace('c', 'median_c')).reset_index()
quant3_cost = joined.groupby('mode_type')[['cost', 'comfort']].quantile(.75) 
quant3_cost = quant3_cost.rename(columns=lambda x: x.replace('c', 'q3_c')).reset_index()

The distribution of records as well as costs and comfort for these categories is as follows:

In [376]:
# combined stats for cost
q4 = """
     SELECT s.mode_type
          , s.cnt
          , s.avg_cost
          , std.std_cost
          , s.min_cost
          , q1.q1_cost
          , q2.median_cost
          , q3.q3_cost
          , s.max_cost
     FROM stats_by_type s
     INNER JOIN std_dev std
     USING (mode_type)
     INNER JOIN quant1_cost q1
     USING (mode_type)
     INNER JOIN quant2_cost q2
     USING (mode_type)
     INNER JOIN quant3_cost q3
     USING (mode_type)
     ;
     """

cost_stats = pysqldf(q4)
cost_stats

,mode_type,cnt,avg_cost,std_cost,min_cost,q1_cost,median_cost,q3_cost,max_cost
0,Walk,6,0.000000,0.000000,0.0,0.00,0.00,0.0000,0.00
1,Public transport,37,3.210811,4.267229,2.0,2.25,2.25,2.2500,25.09
2,Rideshare,146,10.343973,7.324178,0.0,6.00,7.00,12.1725,39.76
3,Taxi,7,20.160000,6.555278,14.0,16.30,17.85,21.6700,33.33


In [377]:
# combined stats for comfort
q5 = """
     SELECT s.mode_type
          , s.cnt
          , s.avg_comfort
          , std.std_comfort
          , s.min_comfort
          , q1.q1_comfort
          , q2.median_comfort
          , q3.q3_comfort
          , s.max_comfort
     FROM stats_by_type s
     INNER JOIN std_dev std
     USING (mode_type)
     INNER JOIN quant1_cost q1
     USING (mode_type)
     INNER JOIN quant2_cost q2
     USING (mode_type)
     INNER JOIN quant3_cost q3
     USING (mode_type)
     ;
     """

comfort_stats = pysqldf(q5)
comfort_stats

,mode_type,cnt,avg_comfort,std_comfort,min_comfort,q1_comfort,median_comfort,q3_comfort,max_comfort
0,Walk,6,3.166667,0.752773,2.0,3.0,3.0,3.75,4.0
1,Public transport,37,3.027027,0.957035,1.0,2.0,3.0,4.00,4.0
2,Rideshare,146,3.856164,0.989530,1.0,3.0,4.0,5.00,5.0
3,Taxi,7,4.142857,1.463850,1.0,4.0,5.0,5.00,5.0
